In [ ]:
!pip install wandb

import os
from glob import glob
import numpy as np
import zipfile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D, Dropout,Activation,MaxPooling2D,Flatten, BatchNormalization, Input
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing import image
from keras.losses import categorical_crossentropy
from keras.callbacks import EarlyStopping
import pandas as pd
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint
import random
from keras import regularizers
from keras.layers import MaxPooling2D, Input
from tensorflow.keras.losses import MeanSquaredError
from sklearn.model_selection import train_test_split
from PIL import Image
import wandb
from wandb.keras import WandbCallback

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.4 MB/s eta 0:00:00


In [ ]:
from keras.models import load_model

In [ ]:
base_model = load_model("m1.keras")


for i, layer in enumerate(base_model.layers):
   print(i, layer.name)


0 input_1
1 conv2d
2 conv2d_1
3 max_pooling2d
4 conv2d_2
5 conv2d_3
6 max_pooling2d_1
7 conv2d_4
8 conv2d_5
9 max_pooling2d_2
10 batch_normalization
11 separable_conv2d
12 batch_normalization_1
13 activation
14 separable_conv2d_1
15 max_pooling2d_3
16 batch_normalization_2
17 separable_conv2d_2
18 batch_normalization_3
19 activation_1
20 separable_conv2d_3
21 max_pooling2d_4
22 flatten
23 dense
24 batch_normalization_4
25 dense_1
26 batch_normalization_5
27 dense_2


In [ ]:
base_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 80, 80, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 80, 80, 128)       1280      
                                                                 
 conv2d_1 (Conv2D)           (None, 80, 80, 256)       295168    
                                                                 
 max_pooling2d (MaxPooling2  (None, 40, 40, 256)       0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 40, 40, 64)        147520    
                                                                 
 conv2d_3 (Conv2D)           (None, 40, 40, 128)       73856     
                                                             

In [ ]:
from tensorflow.keras import layers
x = base_model.layers[22].output

x = layers.Flatten(name="f1")(x)
x = layers.Dense(300, activation='relu',kernel_regularizer=regularizers.L1L2(l1=1e-3, l2=1e-3))(x)
x = layers.BatchNormalization(name="b1")(x)
x = layers.Dropout(0.5)(x)

x = layers.Dense(200, activation='relu',kernel_regularizer=regularizers.L1L2(l1=1e-3, l2=1e-3))(x)
x = layers.BatchNormalization(name="b2")(x)
x = layers.Dropout(0.5)(x)


x = layers.Dense(100, activation='relu',kernel_regularizer=regularizers.L1L2(l1=1e-3, l2=1e-3))(x)
x = layers.BatchNormalization(name="b3")(x)
x = layers.Dropout(0.5)(x)

x = layers.Dense(20, activation='relu',kernel_regularizer=regularizers.L1L2(l1=1e-3, l2=1e-3))(x)
x = layers.BatchNormalization(name="b4")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(1, activation="softmax",kernel_regularizer=regularizers.L1L2(l1=1e-3, l2=1e-3) )(x)
model1 = keras.Model(inputs=base_model.input, outputs=outputs)


for layer in model1.layers[:22]:
  layer.trainable = False
model1.summary()


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 80, 80, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 80, 80, 128)       1280      
                                                                 
 conv2d_1 (Conv2D)           (None, 80, 80, 256)       295168    
                                                                 
 max_pooling2d (MaxPooling2  (None, 40, 40, 256)       0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 40, 40, 64)        147520    
                                                                 
 conv2d_3 (Conv2D)           (None, 40, 40, 128)       73856     
                                                           

In [ ]:
!wget http://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/celeba.zip
with zipfile.ZipFile("celeba.zip","r") as zip_ref:
  zip_ref.extractall("data_faces/")

--2024-04-20 04:18:01--  http://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/celeba.zip
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.193.112, 52.219.121.104, 52.219.113.120, ...
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.193.112|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1443490838 (1.3G) [application/zip]
Saving to: ‘celeba.zip’

celeba.zip          100%[===================>]   1.34G  80.8MB/s    in 20s     

2024-04-20 04:18:21 (69.4 MB/s) - ‘celeba.zip’ saved [1443490838/1443490838]



In [ ]:
with zipfile.ZipFile("myimg.zip","r") as zip_ref:
  zip_ref.extractall("myface/")

In [ ]:
import tensorflow as tf

import pathlib
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
path_to_images = 'data_faces/img_align_celeba/'


df = pd.read_csv("list.csv")

n_images=1600
from_image = 50001
to_image= from_image+n_images
files = [path_to_images+fileName for fileName in df.iloc[from_image:to_image, 0]]
files = tf.data.Dataset.from_tensor_slices(files)
attributes1 = tf.data.Dataset.from_tensor_slices(np.zeros((n_images,1)))
data1 = tf.data.Dataset.zip((files, attributes1))
print(data1)

path = 'myface/myimg'
myFiles = glob(os.path.join(path, '*'))
n_files = len(myFiles)
#myFiles = [os.path.basename(filename) for filename in myFiles]
myFiles = tf.data.Dataset.from_tensor_slices(myFiles[:])
attributes2 = tf.data.Dataset.from_tensor_slices(np.ones((n_files,1)))
data2 = tf.data.Dataset.zip((myFiles,attributes2))
data = data1.concatenate(data2)
data = data.shuffle(data.cardinality())
size_image = 80
def process_file(file_name, attributes):
    image = tf.io.read_file(file_name)
    image = tf.image.decode_jpeg(image, channels=1)
    image = tf.image.resize(image, [size_image, size_image])
    image /= 255.0  #
    return image, attributes

labeled_images = data.map(process_file,num_parallel_calls=tf.data.AUTOTUNE).batch(50).prefetch(tf.data.AUTOTUNE)

dataset = labeled_images
total_size = len(labeled_images)
print(len(labeled_images))
print(len(dataset))

train_size = int(0.8 * total_size)
validation_size = total_size - train_size


train = dataset.take(train_size)
val = dataset.skip(train_size)


<_ZipDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(1,), dtype=tf.float64, name=None))>
66
66


In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
wandb.init(project="ReconocimientoFacial",
          config={"Learning_rate":0.00005,
                 "epochs":50,
                 "batch_size":50,
                 "loss_function":"binary_crossentropy",
                 "optimizer":"Adam",
                  "ConvType":"Block, Sep"
                 })
config = wandb.config
run = wandb.run

model1.compile(loss="binary_crossentropy",optimizer=Adam(learning_rate=0.00005),metrics=['accuracy'])
model1.fit(train,epochs=50,verbose=1,validation_data=val, callbacks=[WandbMetricsLogger()])

run.finish()

Epoch 1/50
 6/52 [==>...........................] - ETA: 5s - loss: 5.8713 - accuracy: 0.4700

52/52 [==============================] - 26s 192ms/step - loss: 5.7655 - accuracy: 0.5192 - val_loss: 5.7546 - val_accuracy: 0.5332
Epoch 2/50
52/52 [==============================] - 10s 200ms/step - loss: 5.5705 - accuracy: 0.5115 - val_loss: 5.5056 - val_accuracy: 0.5246
Epoch 3/50
52/52 [==============================] - 9s 175ms/step - loss: 5.4095 - accuracy: 0.5135 - val_loss: 5.4551 - val_accuracy: 0.5217
Epoch 4/50
52/52 [==============================] - 9s 177ms/step - loss: 5.2520 - accuracy: 0.5081 - val_loss: 5.1546 - val_accuracy: 0.5549
Epoch 5/50
52/52 [==============================] - 9s 181ms/step - loss: 5.0767 - accuracy: 0.5131 - val_loss: 5.0195 - val_accuracy: 0.5000
Epoch 6/50
52/52 [==============================] - 12s 228ms/step - loss: 4.9194 - accuracy: 0.5158 - val_loss: 4.8539 - val_accuracy: 0.5087
Epoch 7/50
52/52 [==============================] - 9s 177ms/step - loss: 4.7676 - accuracy: 0.5208 - val_loss: 4.6951 - val_accuracy: 0.5275
Epoch 8/50
52/

epoch/accuracy,▇▄▅▃▅▇▅█▄▄▆▂▅▅▅▆▆▅▄▆▄▆▄▆▅▆▅▄▅▄▂▅▅▆▆█▅▇▆▁
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,██▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
epoch/val_accuracy,▆▅▅█▄▅▂▆▃▅▆▄▂▁▅▆▅▄▃▂▄▆▁▃▄▄▄▃▅▄▃▅▅▆▃▆▅▄▄▃
epoch/val_loss,▇▇▇▇▆▆█▆▅▅▅▅▄▄▄▄▃▃▃▃▃▄▃▅▂▂▂▅▂▂▂▂▁▁▂▁▁▁▂▁
epoch/accuracy,0.50269
epoch/epoch,49
epoch/learning_rate,5e-05
epoch/loss,1.18376
epoch/val_accuracy,0.50723


In [ ]:
for layer in model1.layers[:22]:
  layer.trainable = True
model1.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 80, 80, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 80, 80, 128)       1280      
                                                                 
 conv2d_1 (Conv2D)           (None, 80, 80, 256)       295168    
                                                                 
 max_pooling2d (MaxPooling2  (None, 40, 40, 256)       0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 40, 40, 64)        147520    
                                                                 
 conv2d_3 (Conv2D)           (None, 40, 40, 128)       73856     
                                                           